# Costos y Beneficios asociados a la implementación del SMI por Comuna (Resultados)

Este notebook refleja los resultados documentados en 'CB_implementacion_SMI_Chile.ipynb' pero sin argumentar los costos ni detallar la forma del cálculo para efectos prácticos de la obtención de resultados.

In [88]:
import pandas as pd 
import numpy as np
import unidecode as ud
from pprint import pprint


df_caracterizacion_distribucion = pd.read_excel("./Para obtener la caracterización de Dx/Salidas/Caracterizacion_Dx_Dic2024_comuna (CNE).xlsx")

# Dolar y UF considerados (a 16 de Oct de 2025, según SII)
usd = 959.36
uf 	= 39_521.26

# Se elimina la columna "Cantidad de alimentadores" que no es necesaria para el análisis (no incluidas en el SMMC de la Norma)
df_caracterizacion_distribucion = df_caracterizacion_distribucion.drop(columns=["Cantidad de alimentadores",
																				"Energía facturada [USD]"])

# Se setea la comuna y el segmento como índice del DataFrame
df_caracterizacion_distribucion = df_caracterizacion_distribucion.set_index(["Comuna", "Segmento"], drop=False)

# Se anualiza la Energía facturada
df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] = (df_caracterizacion_distribucion["Energía promedio mensual [kWh]"] * 12) / 1000  # Conversión a MWh/año

# Se renombran columnas para mayor claridad
df_caracterizacion_distribucion = df_caracterizacion_distribucion.rename(columns={
	"Energía promedio mensual [kWh]": "Energía facturada anual [MWh/año]"
})

# Decisión de la tecnología a ocupar; PLC, Celular, RF, LoRa, etc.
df_implementacion = df_caracterizacion_distribucion.copy()

# Se definen los niveles de densidad que existen en la tabla
niveles_densidad = [
	"EXTREMADAMENTE BAJA",
	"MUY BAJA",
	"BAJA",
	"MEDIA",
	"ALTA"
]

print("Se asigna la tecnología de comunicaciones para cada nivel de densidad")
print("Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'")

# Input de tecnología y armando del mapeo
mapeo_tecnologia = {}

# mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "Celular"
# mapeo_tecnologia["MUY BAJA"] = "Celular"
# mapeo_tecnologia["BAJA"] = "Celular"
# mapeo_tecnologia["MEDIA"] = "Celular"
# mapeo_tecnologia["ALTA"] = "Celular"

# mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "TWACS"
# mapeo_tecnologia["MUY BAJA"] = "TWACS"
# mapeo_tecnologia["BAJA"] = "TWACS"
# mapeo_tecnologia["MEDIA"] = "TWACS"
# mapeo_tecnologia["ALTA"] = "TWACS"

# mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "G3-PLC"
# mapeo_tecnologia["MUY BAJA"] = "G3-PLC"
# mapeo_tecnologia["BAJA"] = "G3-PLC"
# mapeo_tecnologia["MEDIA"] = "G3-PLC"
# mapeo_tecnologia["ALTA"] = "G3-PLC"

# mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "RF-Mesh"
# mapeo_tecnologia["MUY BAJA"] = "RF-Mesh"
# mapeo_tecnologia["BAJA"] = "RF-Mesh"
# mapeo_tecnologia["MEDIA"] = "RF-Mesh"
# mapeo_tecnologia["ALTA"] = "RF-Mesh"

# mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "LoRa"
# mapeo_tecnologia["MUY BAJA"] = "LoRa"
# mapeo_tecnologia["BAJA"] = "LoRa"
# mapeo_tecnologia["MEDIA"] = "LoRa"
# mapeo_tecnologia["ALTA"] = "LoRa"

mapeo_tecnologia["EXTREMADAMENTE BAJA"] = "LoRa"
mapeo_tecnologia["MUY BAJA"] = "LoRa"
mapeo_tecnologia["BAJA"] = "RF-Mesh"
mapeo_tecnologia["MEDIA"] = "G3-PLC"
mapeo_tecnologia["ALTA"] = "G3-PLC"

# Se crea la nueva columna usando la columna 'Densidad'
df_implementacion["Tecnología"] = df_implementacion["Densidad"].map(mapeo_tecnologia)


### Se eliminan los requerimientos de UM para TD
# 0: No se requieren UM para TD
# else: Se requieren UM para TD
despliegue_um_td = 1

if despliegue_um_td == 0:
	df_implementacion["UM 1F para TD"] = 0
	df_implementacion["UM 3F 1S para TD"] = 0
	df_implementacion["UM 3F 2S para TD"] = 0

Se asigna la tecnología de comunicaciones para cada nivel de densidad
Tecnologías posibles: 'Celular', 'TWACS', 'G3-PLC', 'RF-Mesh', 'LoRa'


## Requerimientos sobre la cantidad de equipos necesarios por comuna y segmento

In [89]:
# Se crea el df asociado a una implementación real temporal
crecimiento_anual_demanda = 0.0302  			# Crecimiento anual de la demanda (3.02%)

crecimiento_anual_residencial = 0.0153  		# Crecimiento anual de clientes residenciales (BT) (1.53%)
crecimiento_anual_no_residencial = 0.0094  		# Crecimiento anual de clientes no residenciales (AT) (0.94%)
crecimiento_anual_libredx = 0.0315  			# Crecimiento anual de clientes libre Dx (3.15%)

df_implementacion_1 = df_implementacion.copy()		# Año 1 (2026)
df_implementacion_2 = df_implementacion.copy()		# Año 2 (2027)
df_implementacion_3 = df_implementacion.copy()		# Año 3 (2028)
df_implementacion_4 = df_implementacion.copy()		# Año 4 (2029)
df_implementacion_5 = df_implementacion.copy()		# Año 5 (2030)
df_implementacion_6 = df_implementacion.copy()		# Año 6 (2031)
df_implementacion_7 = df_implementacion.copy()		# Año 7 (2032)
df_implementacion_8 = df_implementacion.copy()		# Año 8 (2033)
df_implementacion_9 = df_implementacion.copy()		# Año 9 (2034)
df_implementacion_10 = df_implementacion.copy()		# Año 10 (2035)
df_implementacion_11 = df_implementacion.copy()		# Año 11 (2036)
df_implementacion_12 = df_implementacion.copy()		# Año 12 (2037)
df_implementacion_13 = df_implementacion.copy()		# Año 13 (2038)
df_implementacion_14 = df_implementacion.copy()		# Año 14 (2039)
df_implementacion_15 = df_implementacion.copy()		# Año 15 (2040)

lista_dfs_implementacion = [df_implementacion,
							df_implementacion_1,
							df_implementacion_2,
							df_implementacion_3,
							df_implementacion_4,
							df_implementacion_5,
							df_implementacion_6,
							df_implementacion_7,
							df_implementacion_8,
							df_implementacion_9,
							df_implementacion_10,
							df_implementacion_11,
							df_implementacion_12,
							df_implementacion_13,
							df_implementacion_14,
							df_implementacion_15]


# Los datos utilizados de energía y cantidad de clientes corresponden al año 2024 (df_implementacion), sin embargo,
# el año base de la implementación es 2025 (df_implementacion_0), salvo para el caso de lo clientes libres el cual si está actualizado a 2025.
factor_crecimiento_cantidad_clientes = np.where(df_implementacion["Segmento"] == 'Residencial', crecimiento_anual_residencial,
											np.where(df_implementacion["Segmento"] == 'No Residencial BT', crecimiento_anual_residencial,
												np.where(df_implementacion["Segmento"] == 'No Residencial AT', crecimiento_anual_no_residencial, 0)))

factor_crecimiento_demanda = np.where(df_implementacion["Segmento"] == 'Residencial', crecimiento_anual_demanda,
								np.where(df_implementacion["Segmento"] == 'No Residencial BT', crecimiento_anual_demanda,
									np.where(df_implementacion["Segmento"] == 'No Residencial AT', crecimiento_anual_demanda, 0)))

# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado (factor compuesto)
lista_dfs_implementacion[0]["Cantidad de clientes"] = ((lista_dfs_implementacion[0]["Cantidad de clientes"]
														* (1 + factor_crecimiento_cantidad_clientes)).round().astype('Int64'))

# Se actualiza la Energía facturada anual según el crecimiento anual esperado (factor compuesto)
lista_dfs_implementacion[0]["Energía facturada anual [MWh/año]"] = (lista_dfs_implementacion[0]["Energía facturada anual [MWh/año]"]
																	* (1 + factor_crecimiento_demanda))


### Se calclula la cantidad de Unidades de Medida (UM) necesarias
for periodo in range(len(lista_dfs_implementacion)):

	# Se agrega la columna de "Cantidad UM 1F" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'Residencial' or row['Segmento'] == 'No Residencial BT':
			if periodo == 2:
				valores.append(row['Cantidad de clientes'])
			elif periodo > 2 and row['Segmento'] == 'Residencial':
				var = row["Cantidad de clientes"] - lista_dfs_implementacion[periodo - 1].loc[(row["Comuna"], "Residencial"), "Cantidad de clientes"]
				valores.append(var)
			elif periodo > 2 and row['Segmento'] == 'No Residencial BT':
				var = row["Cantidad de clientes"] - lista_dfs_implementacion[periodo - 1].loc[(row["Comuna"], "No Residencial BT"), "Cantidad de clientes"]
				valores.append(var)
			else:
				valores.append(0)
		elif row['Segmento'] == 'Red':
			valores.append(row['UM 1F para TD'])
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 1F"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se agrega la columna de "Cantidad UM 3F Directa" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'No Residencial AT':
			if periodo == 2:
				valores.append(row['Cantidad de clientes'])
			elif periodo > 2:
				var = row["Cantidad de clientes"] - lista_dfs_implementacion[periodo - 1].loc[(row["Comuna"], "No Residencial AT"), "Cantidad de clientes"]
				valores.append(var)
			else:
				valores.append(0)
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se agrega la columna de "Cantidad UM 3F Indirecta" según el segmento
	valores = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		
		if row['Segmento'] == 'Red' or row['Segmento'] == 'LibreDx':		# Los clientes libres de bajo consumo si se les implementa UM antes de 2027 según la Norma
			if periodo == 0:
				valores.append(row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'] * 2 + row['Cantidad de clientes'])
			elif row["Segmento"] == "LibreDx":
				var = row["Cantidad de clientes"] - lista_dfs_implementacion[periodo - 1].loc[(row["Comuna"], "LibreDx"), "Cantidad de clientes"]
				valores.append(var)
			else:
				valores.append(0)
		else:
			valores.append(0)

	lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index).round().astype('Int64')

	# Se preparan los datos para el siguiente período (factor compuesto)
	# lista_dfs_implementacion[periodo]["Cantidad de clientes"] = (lista_dfs_implementacion[0]["Cantidad de clientes"]
	# 															   * (1 + crecimiento_anual_residencial) ** periodo).round().astype('Int64')

	if periodo + 1 < len(lista_dfs_implementacion):
		lista_dfs_implementacion[periodo + 1]["UM 1F para TD"] = 0
		lista_dfs_implementacion[periodo + 1]["UM 3F 1S para TD"] = 0
		lista_dfs_implementacion[periodo + 1]["UM 3F 2S para TD"] = 0

		# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado
		factor_crecimiento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'Residencial', crecimiento_anual_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'No Residencial BT', crecimiento_anual_residencial,
								 np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'No Residencial AT', crecimiento_anual_no_residencial,
								np.where(lista_dfs_implementacion[periodo]["Segmento"] == 'LibreDx', crecimiento_anual_libredx,
								0))))
		
		# Se actualizan los valores de "Cantidad de clientes" según el crecimiento anual esperado (factor compuesto)
		lista_dfs_implementacion[periodo + 1]["Cantidad de clientes"] = (((lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																		* (1 + factor_crecimiento)).round().astype('Int64')))

		# Se actualiza la Energía facturada anual según el crecimiento anual esperado (factor compuesto)
		lista_dfs_implementacion[periodo + 1]["Energía facturada anual [MWh/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* (1 + crecimiento_anual_demanda))
		

### Se calclula la cantidad de Unidades Concentradoras (DCU) necesarias
valores = []

for i, row in df_implementacion.iterrows():
	
	if row["Tecnología"] == "G3-PLC" and row["Segmento"] == "Red":
		dcu_requeridas = (row['UM 1F para TD'] + row['UM 3F 1S para TD'] + row['UM 3F 2S para TD'])		# Sujeto a cambios según estimación de TD
		valores.append(dcu_requeridas)
	
	elif row["Tecnología"] == "RF-Mesh" and row["Segmento"] == "Red":
		capacidad = 1000  																								# Capacidad de 1000 equipos por DCU
		cantidad_cliente_comuna = df_implementacion.loc[(row["Comuna"], slice(None)), "Cantidad de clientes"].sum()			
		dcu_requeridas = (cantidad_cliente_comuna // capacidad) + 1  													# Redondeo hacia arriba
		valores.append(dcu_requeridas)

	elif row["Tecnología"] == "LoRa" and row["Segmento"] == "Red":
		if row["Densidad"] == "EXTREMADAMENTE BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "BAJA":
			cobertura_km2 = (15 ** 2) * np.pi 
			dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Rural 15 km
			valores.append(dcu_requeridas) 	
		else:
			cobertura_km2 = (2 ** 2) * np.pi
			dcu_requeridas = np.ceil(row["Superficie efectiva [km2]"] / cobertura_km2)					# Urbano 2 km
			valores.append(dcu_requeridas)
		
	elif row["Tecnología"] == "TWACS" and row["Segmento"] == "Red":										# Caso TWACS: 1 DCU por SPD
		dcu_requeridas = row["Cantidad SPD"]							
		valores.append(dcu_requeridas)

	else:        
		dcu_necesarias = 0								# Caso de Celular y otros segmentos
		valores.append(dcu_necesarias)

df_implementacion["Cantidad DCU"] = pd.Series(valores, index=df_implementacion.index).astype('Int64')

# Se asignan las cantidades de DCU calculadas al año de implementación 2027
df_implementacion_2["Cantidad DCU"] = df_implementacion["Cantidad DCU"]

# TEMPORAL: Se asigna tecnología celular entre el año 2025 y 2027
df_implementacion["Tecnología"] = "Celular"
df_implementacion_1["Tecnología"] = "Celular"

df_implementacion["Cantidad DCU"] = 0
df_implementacion_1["Cantidad DCU"] = 0

# Se asignan 0 DCU para el resto de los años
for periodo in range(3, len(lista_dfs_implementacion)):
	lista_dfs_implementacion[periodo]["Cantidad DCU"] = 0


### Se calcula la cantidad de datos transmitida por DCU
# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')

# 1: Se consideran las exigencias de medición de la Norma SMMC hasta antes del 2034 y después de ella (Artículo 9-8)
# 0: Se consideran las exigencias de medición relajadas para que TWACS pueda operar
exigencias_de_medicion = 1		# Input: 1 o 0 o 2

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024					# MB mensual por cliente
	datos_cliente_3f_zona_baja_previo_2034 = 45.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024					# MB mensual por cliente
	datos_cliente_1f_zona_baja_previo_2034 = 32.5 / 1024		# MB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024						# MB mensual por trafo
	datos_trafo_3f_zona_baja_previo_2034 = 45.7 / 1024			# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024						# MB mensual por trafo
	datos_trafo_1f_zona_baja_previo_2034 = 38.5 / 1024			# MB mensual por trafo

	# Valores que cambian después del 2034
	datos_cliente_3f_zona_baja_post_2034 = 110.2 / 1024			# MB mensual por cliente
	datos_cliente_1f_zona_baja_post_2034 = 71.7 / 1024			# MB mensual por cliente

	datos_trafo_3f_zona_baja_post_2034 = 110.2 / 1024			# MB mensual por trafo
	datos_trafo_1f_zona_baja_post_2034 = 83.7 / 1024			# MB mensual por trafo

else:	# Se consideran las exigencias de medición relajadas para que TWACS pueda operar
	datos_cliente_3f_zona_alta = 110.2 / 1024		# MB mensual por cliente
	datos_cliente_3f_zona_baja = 30.3 / 1024		# MB mensual por cliente (modificado para TWACS) -> cada 6 horas = 32,3, cada 12 horas = 30,9, cada 24 horas = 30,3
	datos_cliente_1f_zona_alta = 71.7 / 1024		# MB mensual por cliente
	datos_cliente_1f_zona_baja = 27.4 / 1024		# MB mensual por cliente (modificado para TWACS) -> cada 6 horas = 28,1, cada 12 horas = 27,6, cada 24 horas = 27,4

	datos_trafo_3f_zona_alta = 110.2 / 1024		# MB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# MB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# MB mensual por trafo

for periodo in range(len(lista_dfs_implementacion)):
	valores = []

	if periodo < 9:		# Hasta antes del 2034

		for i, row in lista_dfs_implementacion[periodo].iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
			cantidad_dcu = lista_dfs_implementacion[2].loc[i, "Cantidad DCU"]

			# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
			if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA") and (row["Tecnología"] != "Celular"):
				total_clientes_3f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_alta = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
									+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
				valores.append(mb_necesarios)

			elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA") and (row["Tecnología"] != "Celular"):
				total_clientes_3f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_baja = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_baja_previo_2034 * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja_previo_2034 * total_clientes_1f_zona_baja
									+ datos_trafo_3f_zona_baja_previo_2034 * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja_previo_2034 * total_trafo_1f_zona_baja) / cantidad_dcu
				valores.append(mb_necesarios)

			else:        
				mb_necesarios = 0								# Caso de Celular, TWACS y otros segmentos
				valores.append(mb_necesarios)

		lista_dfs_implementacion[periodo]["MB Mensuales por DCU"] = 0.0
		filtro = lista_dfs_implementacion[periodo]["Segmento"] == "Red"
		
		lista_dfs_implementacion[periodo].loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index[filtro]).round(4)


	else:		# Después del 2034

		for i, row in lista_dfs_implementacion[periodo].iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
			cantidad_dcu = lista_dfs_implementacion[2].loc[i, "Cantidad DCU"]

			# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
			if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA") and (row["Tecnología"] != "Celular"):
				total_clientes_3f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_alta = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_alta = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
									+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
				valores.append(mb_necesarios)

			elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA") and (row["Tecnología"] != "Celular"):
				total_clientes_3f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
				total_clientes_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

				total_trafo_3f_zona_baja = (lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
											+ lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
				total_trafo_1f_zona_baja = lista_dfs_implementacion[periodo].loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

				mb_necesarios = (datos_cliente_3f_zona_baja_post_2034 * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja_post_2034 * total_clientes_1f_zona_baja
									+ datos_trafo_3f_zona_baja_post_2034 * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja_post_2034 * total_trafo_1f_zona_baja) / cantidad_dcu
				valores.append(mb_necesarios)

			else:        
				mb_necesarios = 0								# Caso de Celular, TWACS y otros segmentos
				valores.append(mb_necesarios)				

		lista_dfs_implementacion[periodo]["MB Mensuales por DCU"] = 0.0
		filtro = lista_dfs_implementacion[periodo]["Segmento"] == "Red"
		
		lista_dfs_implementacion[periodo].loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=lista_dfs_implementacion[periodo].index[filtro]).round(4)

## Costos de inversión y operación por comuna y segmento

In [90]:
df_capex_smi = pd.read_excel("Costos.xlsx", sheet_name="CAPEX", skiprows=4, usecols="B:M")
df_capex_smi = df_capex_smi.set_index(["Tecnología", "Densidad"], drop=False)					   # Se asignan indices

# Cálculo de la inversión
# while True:
# 	try:
# 		descuento_por_volumen = float(input("Descuento por volumen (inversión inicial) [%]: ")) / 100      # Definicion del descuento por volumen
# 		if 0 <= descuento_por_volumen <= 1:
# 			break
# 	except ValueError:
# 		pass

descuento_por_volumen = 0.20      # Definicion del descuento por volumen

for periodo in range(len(lista_dfs_implementacion)):

	inversion_um_1f = []
	inversion_um_3f_dir = []
	inversion_um_3f_ind = []
	inversion_dcu = []
	inversion_sgo = []
	inversion_mdms = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		tecnologia = row["Tecnología"]
		densidad = row["Densidad"]

		costo_um_1f = (df_capex_smi.loc[(tecnologia, densidad), "UM 1F [USD/u]"] * (1 - descuento_por_volumen))
		costo_um_3f_dir = (df_capex_smi.loc[(tecnologia, densidad), "UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
		costo_um_3f_ind = (df_capex_smi.loc[(tecnologia, densidad), "UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
		costo_dcu = (df_capex_smi.loc[(tecnologia, densidad), "DCU [USD/u]"] * (1 - descuento_por_volumen))
		costo_sgo = df_capex_smi.loc[(tecnologia, densidad), "SGO [USD/Medidor]"]
		costo_mdms = df_capex_smi.loc[(tecnologia, densidad), "MDMS [USD/Medidor]"]

		instalacion_um_1f = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 1F [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_um_3f_dir = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 3F Directa [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_um_3f_ind = (df_capex_smi.loc[(tecnologia, densidad), "Instalación UM 3F Indirecta [USD/u]"] * (1 - descuento_por_volumen))
		instalacion_dcu = (df_capex_smi.loc[(tecnologia, densidad), "Instalación DCU [USD/u]"] * (1 - descuento_por_volumen))

		inv_um_1f = (row["Cantidad UM 1F"] * costo_um_1f + row["Cantidad UM 1F"] * instalacion_um_1f)
		inv_um_3f_dir = (row["Cantidad UM 3F Directa"] * costo_um_3f_dir + row["Cantidad UM 3F Directa"] * instalacion_um_3f_dir)
		inv_um_3f_ind = (row["Cantidad UM 3F Indirecta"] * costo_um_3f_ind + row["Cantidad UM 3F Indirecta"] * instalacion_um_3f_ind)
		inv_dcu = (row["Cantidad DCU"] * costo_dcu + row["Cantidad DCU"] * instalacion_dcu)
		inv_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_sgo)
		inv_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * costo_mdms)

		inversion_um_1f.append(inv_um_1f)
		inversion_um_3f_dir.append(inv_um_3f_dir)
		inversion_um_3f_ind.append(inv_um_3f_ind)
		inversion_dcu.append(inv_dcu)
		inversion_sgo.append(inv_sgo)
		inversion_mdms.append(inv_mdms)

	# Se agregan las columnas de inversión por equipo al DataFrame
	lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"] = pd.Series(inversion_um_1f, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"] = pd.Series(inversion_um_3f_dir, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"] = pd.Series(inversion_um_3f_ind, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión DCU [USD]"] = pd.Series(inversion_dcu, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión SGO [USD]"] = pd.Series(inversion_sgo, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"] = pd.Series(inversion_mdms, index=lista_dfs_implementacion[periodo].index)

	# Se suma la inversión total por comuna y segmento
	lista_dfs_implementacion[periodo]["Inversión Total [USD]"] = (lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"]
																	+ lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"]
																	  + lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"]
																	 + lista_dfs_implementacion[periodo]["Inversión DCU [USD]"]
																	+ lista_dfs_implementacion[periodo]["Inversión SGO [USD]"]
																	+ lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"]).round(4)


### Se crea un nuevo DataFrame para almacenar los resultados finales
df_resultado_final = pd.DataFrame()

inversion = [
	"Energía facturada anual [MWh/año]",
	"Cantidad UM 1F",
	"Cantidad UM 3F Directa",
	"Cantidad UM 3F Indirecta",
	"Cantidad DCU",
	"Inversión UM 1F [USD/equipo]",
	"Inversión UM 3F Directa [USD/equipo]",
	"Inversión UM 3F Indirecta [USD/equipo]",
	"Inversión DCU [USD/equipo]",
	"Inversión SGO [USD]",
	"Inversión MDMS [USD]"
]

df_resultado_final["Concepto"] = inversion

for periodo in range(len(lista_dfs_implementacion)):
	
	# Se agregan las inversiones de cada item por periodo
	nuevo_periodo = [
		lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum(),
		lista_dfs_implementacion[periodo]["Cantidad DCU"].sum(),
		lista_dfs_implementacion[periodo]["Inversión UM 1F [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 1F"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión UM 3F Directa [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 3F Directa"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión UM 3F Indirecta [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum() if lista_dfs_implementacion[periodo]["Cantidad UM 3F Indirecta"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión DCU [USD]"].sum() / lista_dfs_implementacion[periodo]["Cantidad DCU"].sum() if lista_dfs_implementacion[periodo]["Cantidad DCU"].sum() > 0 else 0,
		lista_dfs_implementacion[periodo]["Inversión SGO [USD]"].sum(),
		lista_dfs_implementacion[periodo]["Inversión MDMS [USD]"].sum()
	]

	df_resultado_final["Periodo {}".format(periodo)] = nuevo_periodo
	
	lista_dfs_implementacion[periodo].drop(columns=["Inversión UM 1F [USD]", "Inversión UM 3F Directa [USD]", 
													"Inversión UM 3F Indirecta [USD]", "Inversión DCU [USD]", 
													"Inversión SGO [USD]", "Inversión MDMS [USD]"], inplace=True)
	

### Costos de Operacrion y Mantenimiento (O&M)
df_opex_smi = pd.read_excel("Costos.xlsx", sheet_name="OPEX", skiprows=4, usecols="B:G")
df_opex_smi = df_opex_smi.set_index(["Tecnología", "Densidad"], drop=False)			        # Se asignan indices

# Planes de datos, cotización de entel a Dic 2022 (ajustado por IPC, factor=1.12)
plan_10_mb = (1200 / usd) * 1.12  			# Costo mensual en pesos chilenos para 10 MB
plan_10_mb_extra = (100 / usd) * 1.12  		# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base 10 MB
plan_200_mb = (2800 / usd) * 1.12  			# Costo mensual en pesos chilenos para 200 MB
plan_300_mb = (3300 / usd) * 1.12  			# Costo mensual en pesos chilenos para 300 MB
plan_200_300_mb_extra = (50 / usd) * 1.12  	# Costo mensual en pesos chilenos por cada 1 MB extra sobre el plan base 200 MB o 300 MB

for periodo in range(len(lista_dfs_implementacion)):

	# Cálculo de los costos de operación
	comunicaciones = []
	comunicaciones_dcu_sgo = []
	sgo = []
	mdms = []
	mantenimiento_equipos = []

	for i, row in lista_dfs_implementacion[periodo].iterrows():
		tecnologia = row["Tecnología"]
		densidad = row["Densidad"]

		opex_com = df_opex_smi.loc[(tecnologia, densidad), "Comunicaciones [USD/medidor]"] 
		opex_sgo = df_opex_smi.loc[(tecnologia, densidad), "Operación SGO [USD/medidor]"]
		opex_mdms = df_opex_smi.loc[(tecnologia, densidad), "Operación MDMS [USD/medidor]"]
		opex_mantenimiento_equipos = df_opex_smi.loc[(tecnologia, densidad), "Soporte y Mantenimiento Equipos [USD/medidor]"]

		operacion_com = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_com)
		operacion_sgo = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_sgo)
		operacion_mdms = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mdms)
		operacion_mantenimiento_equipos = ((row["Cantidad UM 1F"] + row["Cantidad UM 3F Directa"] + row["Cantidad UM 3F Indirecta"]) * opex_mantenimiento_equipos)

		cantidad_dcu = df_implementacion_2.loc[i, "Cantidad DCU"]
		mb_mensuales = row["MB Mensuales por DCU"]

		if mb_mensuales < 26:                   # Conveniencia económica de plan 10 MB
			operacion_com_dcu_sgo = ((plan_10_mb + round(mb_mensuales - 10) * plan_10_mb_extra) * cantidad_dcu * 12) if mb_mensuales >= 10 else (plan_10_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
		if 26 <= mb_mensuales <= 210:			# Conveniencia económica de plan 200 MB
			operacion_com_dcu_sgo = ((plan_200_mb + round(mb_mensuales - 200) * plan_200_300_mb_extra) * cantidad_dcu * 12) if mb_mensuales > 200 else (plan_200_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)
		if 210 < mb_mensuales:
			operacion_com_dcu_sgo = ((plan_300_mb + round(mb_mensuales - 300) * plan_200_300_mb_extra) * cantidad_dcu * 12) if mb_mensuales > 300 else (plan_300_mb * cantidad_dcu * 12)
			comunicaciones_dcu_sgo.append(operacion_com_dcu_sgo)

		comunicaciones.append(operacion_com)
		sgo.append(operacion_sgo)
		mdms.append(operacion_mdms)
		mantenimiento_equipos.append(operacion_mantenimiento_equipos)

	# Se agregan las columnas de inversión por equipo al DataFrame
	lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"] = pd.Series(comunicaciones, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"] = pd.Series(comunicaciones_dcu_sgo, index=lista_dfs_implementacion[periodo].index).astype("Float64")
	lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"] = pd.Series(sgo, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"] = pd.Series(mdms, index=lista_dfs_implementacion[periodo].index)
	lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"] = pd.Series(mantenimiento_equipos, index=lista_dfs_implementacion[periodo].index)

	if 0 < periodo <= len(lista_dfs_implementacion):
		lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación Comunicaciones [USD/año]"]
		lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación SGO [USD/año]"]
		lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Operación MDMS [USD/año]"]
		lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"] += lista_dfs_implementacion[periodo - 1]["Soporte y Mantenimiento Equipos [USD/año]"]

	# Se suma la inversión total por comuna y segmento
	lista_dfs_implementacion[periodo]["Costos Operacionales Totales [USD/año]"] = (lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"]
																					+ lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"]).round(4)
	

### Se crea un nuevo DataFrame para almacenar los resultados finales de OPEX
df_resultado_final_opex = pd.DataFrame()

operacion = [
	"Operación Comunicaciones [USD/año]",
	"Operación Comunicaciones DCU-SGO [USD/año]",
	"Operación SGO [USD/año]",
	"Operación MDMS [USD/año]",
	"Soporte y Mantenimiento Equipos [USD/año]"
]

df_resultado_final_opex["Concepto"] = operacion

for periodo in range(len(lista_dfs_implementacion)):
	
	# Se agregan los costos operacionales de cada item por periodo
	nuevo_periodo_opex = [
		lista_dfs_implementacion[periodo]["Operación Comunicaciones [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación Comunicaciones DCU-SGO [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación SGO [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Operación MDMS [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Soporte y Mantenimiento Equipos [USD/año]"].sum()
	]

	df_resultado_final_opex["Periodo {}".format(periodo)] = nuevo_periodo_opex

	lista_dfs_implementacion[periodo].drop(columns=["Operación Comunicaciones [USD/año]", "Operación Comunicaciones DCU-SGO [USD/año]", 
													  "Operación SGO [USD/año]", "Operación MDMS [USD/año]", 
													  "Soporte y Mantenimiento Equipos [USD/año]"], inplace=True)


df_resultado_final = pd.concat([df_resultado_final, df_resultado_final_opex], ignore_index=True)

# Se elimina el costo de operación de comunicaciones DCU-SGO en el periodo 0 y 1
df_resultado_final.loc[df_resultado_final["Concepto"] == "Operación Comunicaciones DCU-SGO [USD/año]", "Periodo 0"] = 0
df_resultado_final.loc[df_resultado_final["Concepto"] == "Operación Comunicaciones DCU-SGO [USD/año]", "Periodo 1"] = 0

# Inversión y Operación total por periodo
print("\nConsiderando un descuento por volumen de compra e instalación de UM: {:,.0f}%".format(descuento_por_volumen * 100))
print("----------------------------------------------------------------------------------------")
print("\033[1mPeriodo 0)\033[0m", "Inversión total: M USD {:,.4f}".format(lista_dfs_implementacion[0]["Inversión Total [USD]"].sum() / 1_000_000), 
		  " | Costo Operacional total: M USD {:,.4f}".format(lista_dfs_implementacion[0]["Costos Operacionales Totales [USD/año]"].sum() / 1_000_000))
print("----------------------------------------------------------------------------------------")
for periodo in range(1, len(lista_dfs_implementacion)):
	print("-> \033[1mPeriodo {})\033[0m".format(periodo), "Inversión total: M USD {:,.4f}".format(lista_dfs_implementacion[periodo]["Inversión Total [USD]"].sum() / 1_000_000), 
		  " | Costo Operacional total: M USD {:,.4f}".format(lista_dfs_implementacion[periodo]["Costos Operacionales Totales [USD/año]"].sum() / 1_000_000))


Considerando un descuento por volumen de compra e instalación de UM: 20%
----------------------------------------------------------------------------------------
Periodo 0) Inversión total: M USD 178.0656  | Costo Operacional total: M USD 11.4344
----------------------------------------------------------------------------------------
-> Periodo 1) Inversión total: M USD 0.0055  | Costo Operacional total: M USD 11.4347
-> Periodo 2) Inversión total: M USD 1,425.2639  | Costo Operacional total: M USD 94.1897
-> Periodo 3) Inversión total: M USD 20.9837  | Costo Operacional total: M USD 95.4520
-> Periodo 4) Inversión total: M USD 21.3018  | Costo Operacional total: M USD 96.7336
-> Periodo 5) Inversión total: M USD 21.6262  | Costo Operacional total: M USD 98.0347
-> Periodo 6) Inversión total: M USD 21.9526  | Costo Operacional total: M USD 99.3556
-> Periodo 7) Inversión total: M USD 22.2886  | Costo Operacional total: M USD 100.6967
-> Periodo 8) Inversión total: M USD 22.6277  | Cos

## Beneficios esperados por Comuna y Segmento

In [91]:
### Algunas consideraciones y supuestos para el cálculo de los beneficios:

# PORCENTAJES DE REDUCCIÓN (inputs para el cálculo de beneficios) (FIJOS, NO MODIFICAR)
reduccion_lectura_pedestre 		 = 1.000			# 100%
reduccion_corte_y_repo 			 = 0.810			# ~81%
reduccion_duracion_falla 		 = 0.200			# 5% - 35%
reduccion_atencion_clientes 	 = 0.600			# ~60%
reduccion_infraestructura 	 	 = 0.075			# 5% - 10%

# PORCENTAJES DE REDUCCIÓN (inputs para el cálculo de beneficios) (VARIABLES, MODIFICAR SEGÚN ESCENARIO)
reduccion_pnt_hurto 			 = 0.700			# 50% - 80%
reduccion_pnt_comercial 		 = 0.500			# 40% - 60%
reduccion_precio_de_la_energia 	 = 0.500			# 0% - 79%
reduccion_consumo_residencial 	 = 0.085			# 3% - 20%     (Reducción solo aplicable al 7,5% de todos los clientes)
reduccion_consumo_no_residencial = 0.080			# 2,8% - 15%   (Reducción solo aplicable al 7,5% de todos los clientes)
# En el escenario base, la reducción del precio de la energía debe ser del 59% para un VAN = 0. 
# En el escenario pesimista, la reducción del precio de la energía debe ser del 72,7% para un VAN = 0.
# En el escenario optimista, la reducción del precio de la energía debe ser del 45,1% para un VAN = 0.

#---------------------------------------------------------------------------------------

# Precio de la energía (PNP equivalente nacional, Anexos PNP Julio 2025)
precio_energia_usd_mwh = 93.14   		# USD/MWh

# Compensaciones automáticas e instruidas por la SEC por fallas en el suministro eléctrico (Fuente: Informe SEC Dic 2024)
monto_compensado_anual = (8_744_000_000 + 3_961_000_000) * 1.037 / usd			# USD/año Automáticas + Instruidas (2024)
cantidad_de_compensaciones = 3_257_118 + 1_150_658								# N° Automáticas + Instruidas (2024)

# Costo de falla 2024 para el SEN bajo una profundidad de entre 0-5%
costo_de_falla_2024 = 415.60			# USD/MWh

# Cálculo de la energía no suministrada anual (fuente SEC)
energia_no_suministrada_anual = monto_compensado_anual / (2 * costo_de_falla_2024)	 	# MWh/año


### Beneficio 1) Se calcula el ahorro anual por reducción en la lectura pedestre

# Costo de lectura pedestre por medición (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
costos_por_densidad = {
	"ALTA": 0.20416,					# USD/medición
	"MEDIA": 0.2653,					# USD/medición
	"BAJA": 0.4133,					    # USD/medición
	"MUY BAJA": 0.7136,					# USD/medición
	"EXTREMADAMENTE BAJA": 0.8853		# USD/medición	
}

for periodo in range(len(lista_dfs_implementacion)):

	# Se considera entonces:
	# Medición mensual: costo_lectura * 12
	# Medición bimensual: costo_lectura * 6

	# Costo base por registro (por medición)
	costo_base = lista_dfs_implementacion[periodo]["Densidad"].map(costos_por_densidad)

	# Factor anual según periodicidad: mensual = 12, bimensual = 6
	factor_anual = np.where(lista_dfs_implementacion[periodo]["Tipo de facturación"] == "Mensual", 12,
					np.where(lista_dfs_implementacion[periodo]["Tipo de facturación"] == "Bimensual", 6, np.nan))
	
	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
							   np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	# Nueva columna con costo anual
	lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"] = (costo_base
																				* factor_anual
																				* lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																				* indice_segmento
																				* reduccion_lectura_pedestre).round(4)


### Beneficio 2) Ahorros asociados al corte y reposición remotos de suministro

# Costo promedio por corte e reposición de suministro (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
corte_y_repo = [0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 
				   0.4534, 0.4534, 0.4534, 0.4534, 0.4534, 0.4537, 0.4540, 0.4543]				# USD/cliente-año por año desde 2025 a 2040

for periodo in range(len(lista_dfs_implementacion)):

	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
							   np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	# Ahorro anual por reducción en cortes y reposición remotos de suministro
	lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"] = (lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																						* corte_y_repo[periodo]
																						* reduccion_corte_y_repo
																						* indice_segmento)
	

### Beneficio 3) Se calcula el ahorro anual por reducción del hurto de energía y pérdidas comerciales
	
# Se fija el crecimiento anual compuesto de la demanda (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
crecimiento_anual_pt = 0.0000			# % anual de crecimiento de las pérdidas técnicas (0%)
crecimiento_anual_ph = 0.0000			# % anual de crecimiento del hurto de energía (6.43%)
crecimiento_anual_pc = 0.0000			# % anual de crecimiento de las pérdidas comerciales (0%)

for periodo in range(len(lista_dfs_implementacion)):

	lista_dfs_implementacion[periodo]["Pérdidas de energía [pu]"] = (lista_dfs_implementacion[0]["Pérdidas de energía [pu]"]
																		* (1 + crecimiento_anual_pt) ** periodo)
	lista_dfs_implementacion[periodo]["Hurto de energía [pu]"] = (lista_dfs_implementacion[0]["Hurto de energía [pu]"]
																	* (1 + crecimiento_anual_ph) ** periodo)
	lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"] = (lista_dfs_implementacion[0]["Pérdidas comerciales [pu]"]
																		* (1 + crecimiento_anual_pc) ** periodo)

	# Perdidas técnicas (pt), pérdidas por hurto (ph) y pérdidas comerciales (pc) nominales
	pt = lista_dfs_implementacion[periodo]["Pérdidas de energía [pu]"]
	ph = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"]
	pc = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"]

	# Cálculo de las pérdidas reducidas
	hurto_reducido = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"] * reduccion_pnt_hurto
	comercial_reducido = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"] * reduccion_pnt_comercial

	# Se calcula de esa forma dado que los % están sobre la energía comprada, no la facturada
	lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"] = ((((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																					* hurto_reducido)
																					* precio_energia_usd_mwh)
																			+ (((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																					* comercial_reducido)
																					* precio_energia_usd_mwh))
	

### Beneficio 4) Cantidad de fallas y duración promedio anual (Datos del proceso VAD 2024-2028, 'Beneficios.xlsx')
# cantidad_fallas_anual = 56855     # Fallas/año
# duracion_promedio_anual = 5.62    # Horas/falla

costo_de_falla_2025 = 467.19			# USD/MWh

for periodo in range(len(lista_dfs_implementacion)):

	# Energía no suministrada eliminada por año (MWh/año)
	energia_no_suministrada_eliminada = energia_no_suministrada_anual * reduccion_duracion_falla

	beneficio_neto_compensacion_evitada = energia_no_suministrada_eliminada * costo_de_falla_2025 * 2		# USD/año

	# Se asigna una parte a cada segmento según su participación en las compensaciones totales
	ahorro_por_compensacion_evitada_parcializado = beneficio_neto_compensacion_evitada / ((len(lista_dfs_implementacion[periodo]) / 5) * 3)
	lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"] = np.where(lista_dfs_implementacion[periodo]["Segmento"].isin(["Residencial", 
																																					   "No Residencial BT", 
																																					   "No Residencial AT"]),
																																				ahorro_por_compensacion_evitada_parcializado, 0)
	

### Beneficio 5) Ahorro por reducción del costo de la energía en bloques horarios

bloque_caro_de_energia = 0.5126			# % de la energía consumida que se concentra en el bloque caro (horario 18:00 a 7:59hrs)

energia_lavado_de_ropa = 0.016			# % de la energía consumida que representa al lavado de ropa
energia_secado_de_ropa = 0.062			# % de la energía consumida que representa al secado de ropa
energia_plancha_de_ropa = 0.021			# % de la energía consumida que representa al planchado de ropa
energia_aspiradora = 0.050				# % de la energía consumida que representa al uso de la aspiradora
energia_hervidora_agua = 0.040			# % de la energía consumida que representa al hervido de agua

clientes_con_lavadora = 0.980				# % de los clientes que poseen lavadora
clientes_con_secadora = 0.299				# % de los clientes que poseen secadora
clientes_con_plancha = 0.693				# % de los clientes que poseen plancha
clientes_con_aspiradora = 0.534				# % de los clientes que poseen aspiradora
clientes_con_hervidor = 0.779				# % de los clientes que poseen hervidor eléctrico

energia_trasladada_lavado_de_ropa = energia_lavado_de_ropa * clientes_con_lavadora			# % de la energía trasladada del bloque caro al bloque barato sobre el lavado de ropa
energia_trasladada_secado_de_ropa = energia_secado_de_ropa * clientes_con_secadora			# % de la energía trasladada del bloque caro al bloque barato sobre el secado de ropa
energia_trasladada_plancha_de_ropa = energia_plancha_de_ropa * clientes_con_plancha			# % de la energía trasladada del bloque caro al bloque barato sobre el planchado de ropa
energia_trasladada_aspiradora = energia_aspiradora * clientes_con_aspiradora				# % de la energía trasladada del bloque caro al bloque barato sobre el uso de la aspiradora
energia_trasladada_hervidora_agua = energia_hervidora_agua * clientes_con_hervidor			# % de la energía trasladada del bloque caro al bloque barato sobre el hervido de agua

precio_diferencial = precio_energia_usd_mwh * reduccion_precio_de_la_energia		# USD/MWh

for periodo in range(len(lista_dfs_implementacion)):
	
	# Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1, 0)

	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* indice_segmento
																					* bloque_caro_de_energia
																					* energia_trasladada_lavado_de_ropa
																					* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_secado_de_ropa
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_plancha_de_ropa
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_aspiradora
																						* precio_diferencial)
	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] += (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						* indice_segmento
																						* bloque_caro_de_energia
																						* energia_trasladada_hervidora_agua
																						* precio_diferencial)


### Beneficio 5) Ahorro por gestión del consumo energético
# energia_clientes_mayor_consumo = 0.7392
# energia_trasladada = energia_clientes_mayor_consumo * 0.11

# precio_diferencial = 20 * 1.34			# USD/MWh (ajustado por CPI, 2016-2025)

# for periodo in range(len(lista_dfs_implementacion)):
	
# 	# Indice para aplicar el cálculo solo a los segmentos Residencial y No_Residencial
# 	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
# 						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No_Residencial", 1, 0))

# 	lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
# 																					* indice_segmento
# 																					* energia_trasladada
# 																					* precio_diferencial)
	

### Beneficio 6) Estimación del ahorro anual por reducción del consumo energético

clientes_afectos_a_reduccion = 0.075		# % de clientes que reducen su consumo energético (7.5%)

for periodo in range(len(lista_dfs_implementacion)):

	# reducción anual según segmento: Residendial y No Residencial
	reduccion_consumo = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", reduccion_consumo_residencial,
							np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
								   np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))

	# Se calcula Energía facturada anual [MWh/año] * reducción consumo [%] * precio energía [USD/MWh]
	lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"] = (lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																					* clientes_afectos_a_reduccion
																					* reduccion_consumo 
																					* precio_energia_usd_mwh)
	

### Beneficio 7) Ahorro nacional anual por reducción de emisiones de CO2 (Reducción de consumo y reducción de pérdidas)

# Toneladas de CO2 emitidas por MWh generado (Fuente: Estudio Ministerio del Medio Ambiente, Ver 'Proyecciones_y_Estimaciones.xlsx')
emisiones_co2_por_mwh = [0.2190, 0.2084, 0.1982, 0.1885, 0.1793, 0.1706, 0.1622, 0.1543,
						 0.1468, 0.1396, 0.1328, 0.1263, 0.1202, 0.1143, 0.1087, 0.1034]

# Estimaciones a partir del informe "Estimación del precio social del carbono para la evaluación de la inversión pública en Chile" (Ver 'Proyecciones_y_Estimaciones.xlsx')
costo_social_co2 = [71.11, 78.83, 86.55, 94.27, 102.00, 109.72, 117.44, 125.16, 
					   132.88, 140.60, 148.33, 156.05, 163.77, 171.49, 179.21, 186.93]		# Costo social USD/tCO2 (Fuente: Ministerio de Desarrollo Social) desde 2025 a 2040

for periodo in range(len(lista_dfs_implementacion)):

	# reducción consumo anual según segmento: Residendial y No_Residencial
	reduccion_consumo = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", reduccion_consumo_residencial,
							np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", reduccion_consumo_no_residencial,
								   np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", reduccion_consumo_no_residencial, 0)))

	# Perdidas técnicas (pt), pérdidas por hurto (ph) y pérdidas comerciales (pc) nominales
	pt = lista_dfs_implementacion[periodo]["Pérdidas de energía [pu]"]
	ph = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"]
	pc = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"]

	# Cálculo de las pérdidas reducidas
	hurto_reducido = lista_dfs_implementacion[periodo]["Hurto de energía [pu]"] * reduccion_pnt_hurto
	comercial_reducido = lista_dfs_implementacion[periodo]["Pérdidas comerciales [pu]"] * reduccion_pnt_comercial
	
	lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"] = ((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"]
																						  * clientes_afectos_a_reduccion
																						* reduccion_consumo
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])
																					+ ((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																						* hurto_reducido
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])
																					+ (((lista_dfs_implementacion[periodo]["Energía facturada anual [MWh/año]"] / (1 - pt - ph - pc))
																						* comercial_reducido
																						* emisiones_co2_por_mwh[periodo]
																						* costo_social_co2[periodo])))
	

### Beneficio 8) Ahorros por efecto de la reducción del uso de la atención al cliente de las empresas distribuidoras

# Costo promedio de atención al cliente (Fuente: VAD 2024-2028)
atencion_al_cliente = [2.4662, 2.4623, 2.4576, 2.4540, 2.4494, 2.4440, 2.4387, 2.4335,
					   2.4283, 2.4232, 2.4182, 2.4133, 2.4084, 2.4049, 2.4002, 2.3954]			# USD/cliente-año

for periodo in range(len(lista_dfs_implementacion)):

	# Indice para aplicar el cálculo solo al segmento Regulado (Residencial y No_Residencial)
	indice_segmento = np.where(lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", 1,
						np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial BT", 1,
							   np.where(lista_dfs_implementacion[periodo]["Segmento"] == "No Residencial AT", 1, 0)))

	lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"] = (lista_dfs_implementacion[periodo]["Cantidad de clientes"]
																					* atencion_al_cliente[periodo]
																					* reduccion_atencion_clientes
																					* indice_segmento)
	

### Beneficio 9) Ahorros sobre la inversión evitada en infraestructura de red

# Costo anualizado nacional de inversión en infraestructura de red (Fuente: VAD 2024-2028)
inversion_infraestructura = [199701412, 199847572, 199997331, 202194814, 202666824, 202973599,
							203193675, 203407866, 205082472, 205512944, 205851889, 206191313, 206593527, 
							207198255, 207804753, 208413026]     # USD/año

for periodo in range(len(lista_dfs_implementacion)):

	# Inversión evitada
	inversion_evitada = inversion_infraestructura[periodo] * reduccion_infraestructura

	# Se asigna una parte a cada comuna según su "participación" en la inversión total
	ahorro_por_inversion_evitada_parcializada = inversion_evitada / (len(lista_dfs_implementacion[periodo]) / 5)
	lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"] = np.where(lista_dfs_implementacion[periodo]["Segmento"].isin(["Red"]),
																							ahorro_por_inversion_evitada_parcializada, 0)
	

### Resumen de las cosideraciones y supuestos del análisis
print("Configuración de tecnologías:")
for densidad, tecnologia in mapeo_tecnologia.items():
	print(f" - {densidad}: {tecnologia}")
print("\nConsidera además un descuento por volumen de compra e instalación de UM: {:,.0f}%".format(descuento_por_volumen * 100))
print("-------------------------------------------------------------------------------------\n")

# Beneficio 1) Ahorro global anual por efectos de la telemedición
print(f"Beneficio 1) Considera la reducción de un {reduccion_lectura_pedestre * 100}% de lectura pedestre\n")

# Beneficio 2) Ahorro global anual por efectos de la telemedición
print(f"Beneficio 2) Considera la reducción de un {reduccion_corte_y_repo * 100}% en cortes y reposición de suministro\n")

# Beneficio 3) Ahorro global anual por reducción de las PNT
print(f"Beneficio 3) Considera la reducción de un:",
	   f"\n -> {reduccion_pnt_hurto * 100}% en el hurto de energía", 
	   f"\n -> {reduccion_pnt_comercial * 100}% en las pérdidas comerciales\n")

# Beneficio 4) Ahorro global anual por reducción en la duración promedio de las fallas
print(f"Beneficio 4) Considera la reducción de un {reduccion_duracion_falla * 100}% en la duración promedio de una falla\n")

# Beneficio 5) Ahorro global anual por reducción en el precio de la energía
print(f"Beneficio 5) Considera la reducción de un {reduccion_precio_de_la_energia * 100}% en el precio de la energía\n")

# Beneficio 6) Ahorro global anual por reducción del consumo energético
print("Beneficio 6) Considera la reducción de un:", 
	  f"\n -> {reduccion_consumo_residencial * 100}% en el consumo de energía Residencial",
	  f"\n -> {reduccion_consumo_no_residencial * 100}% en el consumo de energía No Residencial\n")

# Beneficio 8) Ahorro global anual por reducción en la atención al cliente
print(f"Beneficio 8) Considera la reducción de un {reduccion_atencion_clientes * 100}% en el uso de atención al cliente\n")

# Beneficio 9) Ahorro global anual por reducción en la inversión de infraestructura de red
print(f"Beneficio 9) Considera la reducción de un {reduccion_infraestructura * 100}% en la inversión de infraestructura de red\n")
print("-------------------------------------------------------------------------------------")

### Suma de todos los beneficios anuales y creación del DataFrame resumen de beneficios
df_resultado_final_beneficios = pd.DataFrame()

beneficios = [
	"Ahorro en lectura pedestre [USD/año]",
	"Ahorro corte y reposición remoto [USD/año]",
	"Ahorro reducción de PNT [USD/año]",
	"Ahorro por compensación evitada [USD/año]",
	"Ahorro por gestión del consumo [USD/año]",
	"Ahorro reducción del consumo [USD/año]",
	"Ahorro reducción emisiones CO2 [USD/año]",
	"Ahorro atención al cliente [USD/año]",
	"Ahorro por inversión evitada [USD/año]"
]

df_resultado_final_beneficios["Concepto"] = beneficios

for periodo in range(len(lista_dfs_implementacion)):

	nuevo_periodo_beneficios = [
		lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"].sum(),
		lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"].sum()
	]

	df_resultado_final_beneficios["Periodo {}".format(periodo)] = nuevo_periodo_beneficios

	# Suma de todos los beneficios anuales
	lista_dfs_implementacion[periodo]["Beneficio total anual [USD/año]"] = (lista_dfs_implementacion[periodo]["Ahorro en lectura pedestre [USD/año]"]				# Beneficio 1
																			+ lista_dfs_implementacion[periodo]["Ahorro corte y reposición remoto [USD/año]"]		# Beneficio 2
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción de PNT [USD/año]"]				# Beneficio 3
																			+ lista_dfs_implementacion[periodo]["Ahorro por compensación evitada [USD/año]"]		# Beneficio 4
																			+ lista_dfs_implementacion[periodo]["Ahorro por gestión del consumo [USD/año]"]			# Beneficio 5
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción del consumo [USD/año]"]			# Beneficio 6
																			+ lista_dfs_implementacion[periodo]["Ahorro reducción emisiones CO2 [USD/año]"]			# Beneficio 7
																			+ lista_dfs_implementacion[periodo]["Ahorro atención al cliente [USD/año]"]				# Beneficio 8
																			+ lista_dfs_implementacion[periodo]["Ahorro por inversión evitada [USD/año]"])			# Beneficio 9
	
	print(" -> Periodo {}) Beneficio total anual del proyecto: M USD {:,.4f}"
			  .format(periodo, lista_dfs_implementacion[periodo]["Beneficio total anual [USD/año]"].sum() / 1_000_000))
	
df_resultado_final = pd.concat([df_resultado_final, df_resultado_final_beneficios], ignore_index=True)


### Exportación de resultados a un archivo Excel

nombre_archivo = "./Escenarios de despliegue/Resultado_de_Implementacion_SMMC_Mixto.xlsx"
# Se guarda el DataFrame en un archivo Excel nuevo
df_implementacion.to_excel(nombre_archivo, sheet_name="Implementacion_SMMC_Mixto", index=False)

with pd.ExcelWriter(
nombre_archivo,
engine="openpyxl",
mode="a",                	# append (agregar)
if_sheet_exists="replace"   # o "overlay" / "new" según tu versión
) as writer:
	df_resultado_final.to_excel(writer, sheet_name="CB_SMMC_Mixto", index=False)


Configuración de tecnologías:
 - EXTREMADAMENTE BAJA: LoRa
 - MUY BAJA: LoRa
 - BAJA: RF-Mesh
 - MEDIA: G3-PLC
 - ALTA: G3-PLC

Considera además un descuento por volumen de compra e instalación de UM: 20%
-------------------------------------------------------------------------------------

Beneficio 1) Considera la reducción de un 100.0% de lectura pedestre

Beneficio 2) Considera la reducción de un 81.0% en cortes y reposición de suministro

Beneficio 3) Considera la reducción de un: 
 -> 70.0% en el hurto de energía 
 -> 50.0% en las pérdidas comerciales

Beneficio 4) Considera la reducción de un 20.0% en la duración promedio de una falla

Beneficio 5) Considera la reducción de un 50.0% en el precio de la energía

Beneficio 6) Considera la reducción de un: 
 -> 8.5% en el consumo de energía Residencial 
 -> 8.0% en el consumo de energía No Residencial

Beneficio 8) Considera la reducción de un 60.0% en el uso de atención al cliente

Beneficio 9) Considera la reducción de un 7.5% en 

In [92]:
### Capacidad mensual según tecnología de comunicaciones

# Se asignan los resultados del último período (requerimientos máximos) para análisis requerimientos de datos
df_requerimientos = df_implementacion_15.copy()

# Se rellenan los valores 0 de MB Mensuales por DCU con 1 (¡Esto es sólo para considerar los datos de todas las comunas, no tiene impacto en el análisis económico)
df_requerimientos.loc[((df_requerimientos["Cantidad DCU"] == 0) & (df_requerimientos["Segmento"] == "Red")), "Cantidad DCU"] = 1

# Se calcula la cantidad de datos transmitida por DCU (Cálulos realizados en 'Costos.xlsx')

# 1: Se consideran las exigencias de medición de la Norma SMMC hasta antes del 2034 (Artículo 9-8)
# 0: Se consideran las exigencias de medición de la Norma SMMC desde el 2034 en adelante (Artículo 9-8)
exigencias_de_medicion = 1		# Input: 1 o 0

if exigencias_de_medicion == 1:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# kB mensual por cliente
	datos_cliente_3f_zona_baja = 45.7 / 1024		# kB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# kB mensual por cliente
	datos_cliente_1f_zona_baja = 32.5 / 1024		# kB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# kB mensual por trafo
	datos_trafo_3f_zona_baja = 45.7 / 1024		# kB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# kB mensual por trafo
	datos_trafo_1f_zona_baja = 38.5 / 1024		# kB mensual por trafo

else:
	datos_cliente_3f_zona_alta = 110.2 / 1024		# kB mensual por cliente
	datos_cliente_3f_zona_baja = 110.2 / 1024		# kB mensual por cliente
	datos_cliente_1f_zona_alta = 71.7 / 1024		# kB mensual por cliente
	datos_cliente_1f_zona_baja = 71.7 / 1024		# kB mensual por cliente

	datos_trafo_3f_zona_alta = 110.2 / 1024		# kB mensual por trafo
	datos_trafo_3f_zona_baja = 110.2 / 1024		# kB mensual por trafo
	datos_trafo_1f_zona_alta = 83.7 / 1024		# kB mensual por trafo
	datos_trafo_1f_zona_baja = 83.7 / 1024		# kB mensual por trafo

valores = []

for i, row in df_requerimientos.iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"		
	cantidad_dcu = df_requerimientos.loc[i, "Cantidad DCU"]

	# Se suman las cantidades correspondientes de UM enfocadas a clientes y transformadores según la densidad de la zona
	if cantidad_dcu > 0 and (row["Densidad"] == "ALTA" or row["Densidad"] == "MEDIA"):
		total_clientes_3f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_alta = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_alta = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_alta * total_clientes_3f_zona_alta + datos_cliente_1f_zona_alta * total_clientes_1f_zona_alta
								+ datos_trafo_3f_zona_alta * total_trafo_3f_zona_alta + datos_trafo_1f_zona_alta * total_trafo_1f_zona_alta) / cantidad_dcu
		valores.append(mb_exigidos)

	elif cantidad_dcu > 0 and (row["Densidad"] == "BAJA" or row["Densidad"] == "MUY BAJA" or row["Densidad"] == "EXTREMADAMENTE BAJA"):
		total_clientes_3f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["No Residencial AT", "LibreDx"]), "Cantidad de clientes"].sum()
		total_clientes_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Residencial", "No Residencial BT"]), "Cantidad de clientes"].sum()

		total_trafo_3f_zona_baja = (df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 1S para TD"].sum()
									+ df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 3F 2S para TD"].sum())
		total_trafo_1f_zona_baja = df_requerimientos.loc[(row["Comuna"], ["Red"]), "UM 1F para TD"].sum()

		mb_exigidos = (datos_cliente_3f_zona_baja * total_clientes_3f_zona_baja + datos_cliente_1f_zona_baja * total_clientes_1f_zona_baja
								+ datos_trafo_3f_zona_baja * total_trafo_3f_zona_baja + datos_trafo_1f_zona_baja * total_trafo_1f_zona_baja) / cantidad_dcu
		valores.append(mb_exigidos)

	else:        
		mb_exigidos = 0								# Caso de Celular, TWACS y otros segmentos
		valores.append(mb_exigidos)
		
df_requerimientos["MB Mensuales por DCU"] = 0.0
filtro = df_requerimientos["Segmento"] == "Red"

df_requerimientos.loc[filtro, "MB Mensuales por DCU"] = pd.Series(valores, index=df_requerimientos.index[filtro]).round(4)


### Capacidad mensual según tecnología de comunicaciones
capacidad_twacs_por_dcu = ((300 / 8) * 60 * 60 * 24 * 30) / 1_000_000				# 300bps a 97,2MB mensuales por DCU TWACS
capacidad_g3plc_por_dcu = ((100 * 1000 / 8) * 60 * 60 * 24 * 30) / 1_000_000			# 35kbps a 11.340MB mensuales por DCU G3-PLC

capacidad_lora_por_dcu = ((50 * 1000 / 8) * 60 * 60 * 24 * 30) / 1_000_000		# 6,944kbps a 2.249MB mensuales por DCU LoRaWAN
capacidad_rfmesh_por_dcu = ((40 * 1000 / 8) * 60 * 60 * 24 * 30) / 1_000_000		# 40kbps a 12.960MB mensuales por DCU RF-Mesh


# Variables auxiliares para contabilizar las comunas con capacidad suficiente
comunas_con_capacidad_suficiente_alta = 0
comunas_con_capacidad_suficiente_media = 0
comunas_con_capacidad_suficiente_baja = 0
comunas_con_capacidad_suficiente_muy_baja = 0
comunas_con_capacidad_suficiente_extremadamente_baja = 0

# Se recorre la fila "red" de cada comuna
for i, row in df_requerimientos.iloc[4::5].iterrows():			# Recorrer solo las filas de segmento "Red"
	densidad = row["Densidad"]
	mb_mensuales = row["MB Mensuales por DCU"]
	tecnologia = row["Tecnología"]

	if densidad == "ALTA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
		if tecnologia == "LoRa":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_lora_por_dcu else 0
		if tecnologia == "RF-Mesh":
			comunas_con_capacidad_suficiente_alta += 1 if mb_mensuales < capacidad_rfmesh_por_dcu else 0	
	
	elif densidad == "MEDIA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
		if tecnologia == "LoRa":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_lora_por_dcu else 0
		if tecnologia == "RF-Mesh":
			comunas_con_capacidad_suficiente_media += 1 if mb_mensuales < capacidad_rfmesh_por_dcu else 0

	elif densidad == "BAJA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
		if tecnologia == "LoRa":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_lora_por_dcu else 0
		if tecnologia == "RF-Mesh":
			comunas_con_capacidad_suficiente_baja += 1 if mb_mensuales < capacidad_rfmesh_por_dcu else 0

	elif densidad == "MUY BAJA":
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
		if tecnologia == "LoRa":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_lora_por_dcu else 0
		if tecnologia == "RF-Mesh":
			comunas_con_capacidad_suficiente_muy_baja += 1 if mb_mensuales < capacidad_rfmesh_por_dcu else 0

	else:	# EXTREMADAMENTE BAJA
		if tecnologia == "TWACS":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_twacs_por_dcu else 0
		if tecnologia == "G3-PLC":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_g3plc_por_dcu else 0	
		if tecnologia == "LoRa":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_lora_por_dcu else 0
		if tecnologia == "RF-Mesh":
			comunas_con_capacidad_suficiente_extremadamente_baja += 1 if mb_mensuales < capacidad_rfmesh_por_dcu else 0
	
total_comunas_alta = len(df_requerimientos[df_requerimientos['Densidad'] == 'ALTA']) // 5
total_comunas_media = len(df_requerimientos[df_requerimientos['Densidad'] == 'MEDIA']) // 5
total_comunas_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'BAJA']) // 5
total_comunas_muy_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'MUY BAJA']) // 5
total_comunas_extremadamente_baja = len(df_requerimientos[df_requerimientos['Densidad'] == 'EXTREMADAMENTE BAJA']) // 5

# Resultados de capacidad suficiente
for densidad, tecnologia in mapeo_tecnologia.items():
	print(f" - {densidad}: {tecnologia}")
print("----------------------------------------------------------------------")
print("Capacidad mensual según tecnología de comunicaciones:")
print(f" - Comunas con densidad ALTA y capacidad suficiente: {comunas_con_capacidad_suficiente_alta} de {total_comunas_alta}, ({(comunas_con_capacidad_suficiente_alta / total_comunas_alta) * 100:.2f}%)")
print(f" - Comunas con densidad MEDIA y capacidad suficiente: {comunas_con_capacidad_suficiente_media} de {total_comunas_media}, ({(comunas_con_capacidad_suficiente_media / total_comunas_media) * 100:.2f}%)")
print(f" - Comunas con densidad BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_baja} de {total_comunas_baja}, ({(comunas_con_capacidad_suficiente_baja / total_comunas_baja) * 100:.2f}%)")
print(f" - Comunas con densidad MUY BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_muy_baja} de {total_comunas_muy_baja}, ({(comunas_con_capacidad_suficiente_muy_baja / total_comunas_muy_baja) * 100:.2f}%)")
print(f" - Comunas con densidad EXTREMADAMENTE BAJA y capacidad suficiente: {comunas_con_capacidad_suficiente_extremadamente_baja} de {total_comunas_extremadamente_baja}, ({(comunas_con_capacidad_suficiente_extremadamente_baja / 
																																											 total_comunas_extremadamente_baja) * 100:.2f}%)")

 - EXTREMADAMENTE BAJA: LoRa
 - MUY BAJA: LoRa
 - BAJA: RF-Mesh
 - MEDIA: G3-PLC
 - ALTA: G3-PLC
----------------------------------------------------------------------
Capacidad mensual según tecnología de comunicaciones:
 - Comunas con densidad ALTA y capacidad suficiente: 31 de 31, (100.00%)
 - Comunas con densidad MEDIA y capacidad suficiente: 42 de 42, (100.00%)
 - Comunas con densidad BAJA y capacidad suficiente: 49 de 49, (100.00%)
 - Comunas con densidad MUY BAJA y capacidad suficiente: 101 de 101, (100.00%)
 - Comunas con densidad EXTREMADAMENTE BAJA y capacidad suficiente: 107 de 107, (100.00%)


In [93]:
### Calculo sobre los costos netos de inversión y operación por tipo de densidad
costos_por_densidad = df_implementacion.groupby(["Tecnología", "Densidad"]).agg({
	"Inversión Total [USD]": "sum",
	"Costos Operacionales Totales [USD/año]": "sum"
}).reset_index()

for i, row in costos_por_densidad.iterrows():
	densidad = row["Densidad"]
	inversion_total = row["Inversión Total [USD]"]
	costos_operacionales = row["Costos Operacionales Totales [USD/año]"]

	if densidad == "ALTA":
		tecnología_alta = row["Tecnología"]
		inversion_promedio_alta = inversion_total / total_comunas_alta / 1_000_000		# En millones de USD
		costos_operacionales_promedio_alta = costos_operacionales / total_comunas_alta / 1_000_000
	
	elif densidad == "MEDIA":
		tecnología_media = row["Tecnología"]
		inversion_promedio_media = inversion_total / total_comunas_media / 1_000_000		# En millones de USD
		costos_operacionales_promedio_media = costos_operacionales / total_comunas_media / 1_000_000	
	
	elif densidad == "BAJA":
		tecnología_baja = row["Tecnología"]
		inversion_promedio_baja = inversion_total / total_comunas_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_baja = costos_operacionales / total_comunas_baja / 1_000_000
		
	elif densidad == "MUY BAJA":
		tecnología_muy_baja = row["Tecnología"]
		inversion_promedio_muy_baja = inversion_total / total_comunas_muy_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_muy_baja = costos_operacionales / total_comunas_muy_baja / 1_000_000
		
	else:	# EXTREMADAMENTE BAJA
		tecnología_extremadamente_baja = row["Tecnología"]
		inversion_promedio_extremadamente_baja = inversion_total / total_comunas_extremadamente_baja / 1_000_000		# En millones de USD
		costos_operacionales_promedio_extremadamente_baja = costos_operacionales / total_comunas_extremadamente_baja / 1_000_000

print("Costos netos promedio por tipo de densidad:")
print("----------------------------------------------------------------------")
print(f"Tecnología densidad ALTA: {tecnología_alta} | Inversión promedio [MUSD] {inversion_promedio_alta:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_alta:,.4f}")
print(f"Tecnología densidad MEDIA: {tecnología_media} | Inversión promedio [MUSD] {inversion_promedio_media:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_media:,.4f}")
print(f"Tecnología densidad BAJA: {tecnología_baja} | Inversión promedio [MUSD] {inversion_promedio_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_baja:,.4f}")
print(f"Tecnología densidad MUY BAJA: {tecnología_muy_baja} | Inversión promedio [MUSD] {inversion_promedio_muy_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_muy_baja:,.4f}")
print(f"Tecnología densidad EXTREMADAMENTE BAJA: {tecnología_extremadamente_baja} | Inversión promedio [MUSD] {inversion_promedio_extremadamente_baja:,.2f} | Costos operacionales promedio [MUSD/año] {costos_operacionales_promedio_extremadamente_baja:,.4f}")

Costos netos promedio por tipo de densidad:
----------------------------------------------------------------------
Tecnología densidad ALTA: Celular | Inversión promedio [MUSD] 1.00 | Costos operacionales promedio [MUSD/año] 0.0755
Tecnología densidad MEDIA: Celular | Inversión promedio [MUSD] 0.82 | Costos operacionales promedio [MUSD/año] 0.0633
Tecnología densidad BAJA: Celular | Inversión promedio [MUSD] 0.55 | Costos operacionales promedio [MUSD/año] 0.0329
Tecnología densidad MUY BAJA: Celular | Inversión promedio [MUSD] 0.44 | Costos operacionales promedio [MUSD/año] 0.0272
Tecnología densidad EXTREMADAMENTE BAJA: Celular | Inversión promedio [MUSD] 0.38 | Costos operacionales promedio [MUSD/año] 0.0194


In [94]:
### Sobre el cálculo de la cantidad de brigadas necesarias para la instalación de UM
# Se declaran las productividades diarias promedio en función de la densidad y el tipo de equipo
prod_um1f_alta = 10			# UM/día en zona alta
prod_um3fd_alta = 8			# UM/día en zona alta
prod_um3fi_alta = 3			# UM/día en zona alta

prod_um1f_media = 8			# UM/día en zona media
prod_um3fd_media = 6		# UM/día en zona media
prod_um3fi_media = 3		# UM/día en zona media

prod_um1f_baja = 6			# UM/día en zona baja
prod_um3fd_baja = 4			# UM/día en zona baja
prod_um3fi_baja = 2			# UM/día en zona baja

prod_um1f_muy_baja = 4		# UM/día en zona muy baja
prod_um3fd_muy_baja = 3		# UM/día en zona muy baja	
prod_um3fi_muy_baja = 2		# UM/día en zona muy baja

prod_um1f_extremadamente_baja = 3		# UM/día en zona extremadamente baja
prod_um3fd_extremadamente_baja = 2		# UM/día en zona extremadamente baja	
prod_um3fi_extremadamente_baja = 1		# UM/día en zona extremadamente baja

objetivo_del_despliegue = 10		# Años para completar el despliegue

brigadas_zona_alta = 0
brigadas_zona_media = 0
brigadas_zona_baja = 0
brigadas_zona_muy_baja = 0
brigadas_zona_extremadamente_baja = 0

for i, row in df_implementacion.iterrows():
	densidad = row["Densidad"]
	um1f = row["Cantidad UM 1F"] / (objetivo_del_despliegue * 260)
	um3fd = row["Cantidad UM 3F Directa"] / (objetivo_del_despliegue * 260)
	um3fi = row["Cantidad UM 3F Indirecta"] / (objetivo_del_despliegue * 260)
	
	if densidad == "ALTA":
		brigadas_um1f = um1f / prod_um1f_alta
		brigadas_um3fd = um3fd / prod_um3fd_alta
		brigadas_um3fi = um3fi / prod_um3fi_alta
		brigadas_zona_alta += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "MEDIA":
		brigadas_um1f = um1f / prod_um1f_media
		brigadas_um3fd = um3fd / prod_um3fd_media
		brigadas_um3fi = um3fi / prod_um3fi_media
		brigadas_zona_media += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "BAJA":
		brigadas_um1f = um1f / prod_um1f_baja
		brigadas_um3fd = um3fd / prod_um3fd_baja
		brigadas_um3fi = um3fi / prod_um3fi_baja
		brigadas_zona_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	elif densidad == "MUY BAJA":
		brigadas_um1f = um1f / prod_um1f_muy_baja
		brigadas_um3fd = um3fd / prod_um3fd_muy_baja
		brigadas_um3fi = um3fi / prod_um3fi_muy_baja
		brigadas_zona_muy_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

	else:	# EXTREMADAMENTE BAJA
		brigadas_um1f = um1f / prod_um1f_extremadamente_baja
		brigadas_um3fd = um3fd / prod_um3fd_extremadamente_baja
		brigadas_um3fi = um3fi / prod_um3fi_extremadamente_baja
		brigadas_zona_extremadamente_baja += brigadas_um1f + brigadas_um3fd + brigadas_um3fi

total_brigadas = (brigadas_zona_alta + brigadas_zona_media + brigadas_zona_baja
					+ brigadas_zona_muy_baja + brigadas_zona_extremadamente_baja)

print("Cantidad de brigadas necesarias para la instalación de UM en un plazo de {} años: {:,.2f} brigadas".format(objetivo_del_despliegue, total_brigadas))

Cantidad de brigadas necesarias para la instalación de UM en un plazo de 10 años: 45.56 brigadas


In [95]:
### Para obtener la energía promedio por unidad de cliente al año
df_energía_por_cliente = pd.DataFrame()
año = 2025

for periodo in range(len(lista_dfs_implementacion)):
	total_energia_residencial = lista_dfs_implementacion[periodo].loc[lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", "Energía facturada anual [MWh/año]"].sum()
	total_clientes_residencial = lista_dfs_implementacion[periodo].loc[lista_dfs_implementacion[periodo]["Segmento"] == "Residencial", "Cantidad de clientes"].sum()
	energia_promedio_residencial = (total_energia_residencial * 1_000) / total_clientes_residencial / 12		# kWh/cliente-mes

	nueva_fila = {
		"Periodo": periodo,
		"Año": año + periodo,
		"Energía promedio residencial [kWh/cliente-mes]": energia_promedio_residencial.round(2)
	}

	df_energía_por_cliente = pd.concat([df_energía_por_cliente, pd.DataFrame([nueva_fila])], ignore_index=True)	

df_energía_por_cliente

,Periodo,Año,Energía promedio residencial [kWh/cliente-mes]
0,0,2025,187.74
1,1,2026,190.49
2,2,2027,193.29
3,3,2028,196.12
4,4,2029,199.00
5,5,2030,201.92
6,6,2031,204.89
7,7,2032,207.89
8,8,2033,210.94
9,9,2034,214.04
